In [1]:
from mbmvpa.utils.example_utils import load_example_data
from mbmvpa.preprocessing.preprocess import DataPreprocessor
from mbmvpa.data.loader import BIDSDataLoader
from mbmvpa.models.mvpa_general import MVPA_CV
from mbmvpa.models.elasticnet import MVPA_ElasticNet
from mbmvpa.utils.report import build_elasticnet_report_functions
from pathlib import Path

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


In [2]:
root = load_example_data("tom")

data load success! (tom)


## 1. Preprocess data

In [3]:
# Computational model and user-defined functions

dm_model = 'ra_prospect'

def example_adjust(row):
    ## rename data in a row to the name which can match hbayesdm.ra_prospect requirements ##
    row["gamble"] = 1 if row["respcat"] == 1 else 0
    row["cert"] = 0
    return row

def example_filter(row):
    # include all trial data
    return True

def example_latent(row, param_dict):
    ## calculate subjectives utility for choosing Gamble over Safe option
    ## prospect theory with loss aversion and risk aversion is adopted
    modulation = (row["gain"] ** param_dict["rho"]) - (param_dict["lambda"] * (row["loss"] ** param_dict["rho"]))
    row["modulation"] = modulation
    return row

In [4]:
# Define & run preprocessor 

preprocessor = DataPreprocessor(bids_layout=root,
                               adjust_function=example_adjust,
                               filter_function=example_filter,
                               latent_function=example_latent,
                               dm_model=dm_model,
                               mask_threshold=2.58,
                               standardize=True,
                               confounds=[],
                               high_pass=1/128,
                               detrend=False,
                               smoothing_fwhm=6, 
                               zoom=(2,2,2),
                               n_core=4)

INFO: start loading BIDSLayout
INFO: fMRIPrep is loaded
INFO: task name is not designated. find most common task name
INFO: selected task_name is mixedgamblestask
INFO: MB-MVPA is loaded
INFO: 6 file(s) in Original & fMRIPrep.


In [5]:
preprocessor.preprocess(overwrite=True)

AttributeError: 'VoxelFeatureGenerator' object has no attribute 'feature_name'

In [ ]:
preprocessor.summary()

## 2. load data

In [ ]:
loader = BIDSDataLoader(layout=root)
X_dict,y_dict = loader.get_data(subject_wise=True)
voxel_mask = loader.get_voxel_mask()

## 3. Run MVPA model

In [ ]:
model = MVPA_ElasticNet(alpha=0.001,
                         n_samples=50000,
                         shuffle=True,
                         max_lambda=50,
                         min_lambda_ratio=1e-4,
                         lambda_search_num=100,
                         n_jobs=16,
                         n_splits=5)

In [ ]:
report_function_dict = build_elasticnet_report_functions(voxel_mask,
                                                         confidence_interval=.99,
                                                         n_coef_plot=150,
                                                         task_name='unnamed',
                                                         map_type='z',
                                                         sigma=1
                                                         )

In [ ]:
report_path = "example_tom_prospect"
Path(report_path).mkdir(exist_ok=True)

model_cv = MVPA_CV(X_dict,
                    y_dict,
                    model,
                    model_param_dict={},
                    method='5-fold',
                    n_cv_repeat=2,
                    cv_save=True,
                    cv_save_path=report_path,
                    task_name="dd",
                    report_function_dict=report_function_dict)


In [ ]:
model_cv.run()